In [1]:
from __future__ import print_function
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
## Disable tf future deprecated messages
import logging
logging.getLogger('tensorflow').disabled = True
## Disable tf CUDA messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

%matplotlib inline

import sys
import random
import numpy as np
import pandas as pd

import utils
import correction
from models.dense import *
from models.gain import gain
from models.soft_impute import SoftImpute
from models.sinkhorn import OTimputer
from models.mida import mida

import scipy.stats

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer, SimpleImputer

import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
dtype = torch.cuda.FloatTensor

## Repeat all experiments for 10 runs

In [2]:
n_runs = 10

## Load data

In [3]:
data_missing, missing_mask, y = utils.load_miocardial()
## Inject noise
data_missing = utils.inject_noise(data_missing.copy(), noise_rate=.15)
## Replace missing values locations by 0
data_missing = data_missing * missing_mask

Dataset shape: (1700, 110)
5.62% missing data
Class distribution: (array([0, 1]), array([1429,  271]))


## Run OUR METHOD

In [4]:
params = {
    'nb_batches': 10,
    'reg_noise_std': .03,
    'net_input': 'data_corrupted',
    'net_params': {
        'input_channels':1,
        'output_channels':1,
        'channels_skip':4,
        'down_layers':[(24, 7, 2),
                       (46, 9, 1),
                       (96, 9, 5),
                       (96, 11, 1)],
        'need_sigmoid':True,
        'need_bias':True,
        'pad':'zero',
        'downsample_mode':'stride',
        'upsample_mode':'nearest',
        'act_fun':'LeakyReLU',
        'need1x1_up':True
    },# or a list containing layers sizes
    'adam_lr': .0001,
    'adam_weight_decay': 0.,
}

ours_accs, ours_aucs = [], []
for i in range(n_runs):
    ours_correction = correction.run(data_missing, 501, params, y=y, missing_mask=missing_mask, seed=i)
    ## Since our method returns 2 imputations select the best one
    scores_raw = utils.get_scores(ours_correction['raw_out'], y)
    acc = scores_raw['test_balanced_accuracy']
    auc = scores_raw['test_roc_auc_ovo']
    acc = acc.mean()
    acc_std = acc.std()
    auc = auc.mean()
    auc_std = auc.std()
    if 'masked_out' in ours_correction:
        scores_masked = utils.get_scores(ours_correction['masked_out'], y)
        acc_masked = scores_masked['test_balanced_accuracy']
        auc_masked = scores_masked['test_roc_auc_ovo']
        if auc_masked.mean() > auc:
            acc = acc_masked.mean()
            acc_std = acc_masked.std()
            auc = auc_masked.mean()
            auc_std = auc_masked.std()
    print(f'OUR METHOD RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    ours_accs.append(acc.mean())
    ours_aucs.append(auc.mean())
print(f'\nOUR METHOD GLOBAL - acc: {round(np.array(ours_accs).mean() * 100, 2)}% +- {round(np.array(ours_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(ours_aucs).mean() * 100, 2)}% +- {round(np.array(ours_aucs).std() * 100, 2)}%')

Ite 00000 - 6.28 sec - Loss 0.213609 - ACC 72.60% - ACC Mean 72.60% - AUC 77.26% - AUC Mean 77.26% - Deter 000
Ite 00050 - 0.92 sec - Loss 0.100613 - ACC 71.23% - ACC Mean 73.05% - AUC 77.58% - AUC Mean 77.83% - Deter 045
Ite 00055 - 0.96 sec - Loss 0.096516 - ACC 73.02% - ACC Mean 73.07% - AUC 77.54% - AUC Mean 77.78% - Deter 050
Early stop ite 55, rollback to correction of ite 5, whith acc of 73.08% and auc of 80.09%
OUR METHOD RUN 1/10 - acc: 73.08% +- 0.0% - auc: 80.09% +- 0.0%
Ite 00000 - 0.88 sec - Loss 0.223108 - ACC 73.62% - ACC Mean 73.62% - AUC 78.52% - AUC Mean 78.52% - Deter 000
Ite 00050 - 1.05 sec - Loss 0.114689 - ACC 73.42% - ACC Mean 73.48% - AUC 77.62% - AUC Mean 78.45% - Deter 014
Ite 00086 - 1.07 sec - Loss 0.091285 - ACC 72.32% - ACC Mean 73.43% - AUC 77.00% - AUC Mean 77.13% - Deter 050
Early stop ite 86, rollback to correction of ite 36, whith acc of 74.02% and auc of 81.02%
OUR METHOD RUN 2/10 - acc: 74.02% +- 0.0% - auc: 81.02% +- 0.0%
Ite 00000 - 0.82 sec - Lo

## Run Median, Mean and KNN

In [5]:
data_missing_nans = np.where(missing_mask, data_missing, np.nan)

median_imputer = SimpleImputer(missing_values=np.nan, strategy='median')
mean_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
knn_imputer = KNNImputer(n_neighbors=10, weights='uniform')

median_completed = median_imputer.fit_transform(data_missing_nans)
mean_completed = mean_imputer.fit_transform(data_missing_nans)
knn_completed = knn_imputer.fit_transform(data_missing_nans)

## All runs would be the same since deterministic methods
scores = utils.get_scores(median_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MEDIAN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
median_accs = acc
median_aucs = auc

scores = utils.get_scores(mean_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MEAN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mean_accs = acc
mean_aucs = auc

scores = utils.get_scores(knn_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'KNN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
knn_accs = acc
knn_aucs = auc

MEDIAN - acc: 62.6% +- 0.0% - auc: 70.09% +- 0.0%
MEAN - acc: 74.61% +- 0.0% - auc: 79.0% +- 0.0%
KNN - acc: 67.29% +- 0.0% - auc: 75.38% +- 0.0%


## Run GAIN

In [6]:
gain_parameters = {'batch_size': 128,
                   'hint_rate': .9,
                   'alpha': 100,
                   'iterations': 5000}

gain_accs, gain_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputed = gain(data_missing_nans, gain_parameters)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nGAIN RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    gain_accs.append(acc.mean())
    gain_aucs.append(auc.mean())
print(f'GAIN GLOBAL - acc: {round(np.array(gain_accs).mean() * 100, 2)}% +- {round(np.array(gain_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(gain_aucs).mean() * 100, 2)}% +- {round(np.array(gain_aucs).std() * 100, 2)}%')

100%|██████████| 5000/5000 [00:49<00:00, 100.28it/s]



GAIN RUN 1/10 - acc: 72.12% +- 2.06% - auc: 78.48% +- 2.18%



100%|██████████| 5000/5000 [00:48<00:00, 102.79it/s]



GAIN RUN 2/10 - acc: 71.69% +- 3.13% - auc: 78.92% +- 3.34%



100%|██████████| 5000/5000 [00:49<00:00, 101.59it/s]


...


GAIN RUN 8/10 - acc: 70.46% +- 2.62% - auc: 75.29% +- 3.19%



100%|██████████| 5000/5000 [01:00<00:00, 82.43it/s]



GAIN RUN 9/10 - acc: 70.49% +- 3.63% - auc: 77.0% +- 2.47%



100%|██████████| 5000/5000 [01:05<00:00, 76.31it/s]



GAIN RUN 10/10 - acc: 70.14% +- 3.71% - auc: 75.7% +- 4.45%

GAIN GLOBAL - acc: 70.98% +- 0.7% - auc: 77.28% +- 1.4%


## Run MICE

In [7]:
imputer = IterativeImputer()
imputed = imputer.fit_transform(data_missing_nans)
## All runs would be the same since deterministic method
scores = utils.get_scores(imputed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MICE - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mice_accs = acc
mice_aucs = auc

MICE - acc: 73.4% +- 0.0% - auc: 79.58% +- 0.0%


## Run MISSFOREST

In [8]:
miss_accs, miss_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = MissForest(n_jobs=-1, random_state=i)
    imputed = imputer.fit_transform(data_missing_nans)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nMISSFOREST RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    miss_accs.append(acc.mean())
    miss_aucs.append(auc.mean())
print(f'\nMISSFOREST GLOBAL - acc: {round(np.array(miss_accs).mean() * 100, 2)}% +- {round(np.array(miss_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(miss_aucs).mean() * 100, 2)}% +- {round(np.array(miss_aucs).std() * 100, 2)}%')

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6

MISSFOREST RUN 1/10 - acc: 72.44% +- 2.12% - auc: 76.97% +- 1.39%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4

MISSFOREST RUN 2/10 - acc: 71.36% +- 1.27% - auc: 77.08% +- 2.01%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5

MISSFOREST RUN 3/10 - acc: 72.07% +- 1.15% - auc: 76.34% +- 3.01%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6

MISSFOREST RUN 4/10 - acc: 72.9% +- 2.35% - auc: 77.84% +- 3.07%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4

MISSFOREST RUN 5/10 - acc: 72.83% +- 1.88% - auc: 77.57% +- 2.32%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3

MISSFOREST RUN 6/10 - acc: 72.79% +- 2.24% - auc: 76.13% +- 2.96%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6

MISSFOREST RUN 7/10 - acc: 71.94% +- 2.78% - auc: 78.21% +-

## Run SOFTIMPUTE

In [9]:
soft_accs, soft_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = SoftImpute(random_state=i)
    imputer.fit(data_missing_nans)
    imputed = imputer.predict(data_missing_nans)
    imputed = np.where(missing_mask, data_missing, imputed)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'SOFTIMPUTE RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    soft_accs.append(acc.mean())
    soft_aucs.append(auc.mean())
print(f'\nSOFTIMPUTE GLOBAL - acc: {round(np.array(soft_accs).mean() * 100, 2)}% +- {round(np.array(soft_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(soft_aucs).mean() * 100, 2)}% +- {round(np.array(soft_aucs).std() * 100, 2)}%')

SOFTIMPUTE RUN 1/10 - acc: 73.38% +- 4.67% - auc: 79.42% +- 2.13%
SOFTIMPUTE RUN 2/10 - acc: 72.61% +- 3.08% - auc: 75.75% +- 2.53%
SOFTIMPUTE RUN 3/10 - acc: 71.83% +- 3.74% - auc: 77.08% +- 2.81%
SOFTIMPUTE RUN 4/10 - acc: 73.22% +- 2.07% - auc: 78.3% +- 2.18%
SOFTIMPUTE RUN 5/10 - acc: 70.95% +- 3.25% - auc: 77.56% +- 2.73%
SOFTIMPUTE RUN 6/10 - acc: 73.11% +- 2.11% - auc: 77.38% +- 2.68%
SOFTIMPUTE RUN 7/10 - acc: 72.66% +- 4.91% - auc: 80.19% +- 1.66%
SOFTIMPUTE RUN 8/10 - acc: 73.02% +- 2.74% - auc: 78.09% +- 3.06%
SOFTIMPUTE RUN 9/10 - acc: 72.09% +- 3.87% - auc: 74.5% +- 4.02%
SOFTIMPUTE RUN 10/10 - acc: 72.63% +- 3.26% - auc: 77.82% +- 3.0%

SOFTIMPUTE GLOBAL - acc: 72.55% +- 0.71% - auc: 77.61% +- 1.55%


## Run SINKHORN

In [10]:
sink_accs, sink_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = OTimputer(niter=500)
    imputed = imputer.fit_transform(data_missing_nans).cpu().detach().numpy()
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nSINKHORN RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    sink_accs.append(acc.mean())
    sink_aucs.append(auc.mean())
print(f'\nSINKHORN GLOBAL - acc: {round(np.array(sink_accs).mean() * 100, 2)}% +- {round(np.array(sink_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(sink_aucs).mean() * 100, 2)}% +- {round(np.array(sink_aucs).std() * 100, 2)}%')

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 5.4601

SINKHORN RUN 1/10 - acc: 71.91% +- 2.21% - auc: 78.93% +- 1.77%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 5.3284

SINKHORN RUN 2/10 - acc: 71.97% +- 3.12% - auc: 77.7% +- 3.27%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 5.4594

SINKHORN RUN 3/10 - acc: 72.51% +- 1.84% - auc: 78.19% +- 3.01%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 5.4397

SINKHORN RUN 4/10 - acc: 71.3% +- 4.62% - auc: 78.95% +- 2.63%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 5.5366

SINKHORN RUN 5/10 - acc: 72.98% +- 2.58% - auc: 78.15% +- 2.57%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 5.4382

SINKHORN RUN 6/10 - acc: 72.41% +- 2.43% - auc: 77.2% +- 2.31%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 5.5303

SINKHORN RUN 7/10 - acc: 73.14% +- 1.94% - auc: 77.22% +- 1.61%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 5.6527

SINKHORN RUN 8/10 - acc: 72.24% +- 4.05% - auc: 77.1

## Run MIDA

In [11]:
mida_accs, mida_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputed = mida(data_missing_nans, num_layers=2, num_epochs=1500)
    imputed = np.where(missing_mask, data_missing, imputed)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nMIDA RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    mida_accs.append(acc.mean())
    mida_aucs.append(auc.mean())
print(f'\nMIDA GLOBAL - acc: {round(np.array(mida_accs).mean() * 100, 2)}% +- {round(np.array(mida_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(mida_aucs).mean() * 100, 2)}% +- {round(np.array(mida_aucs).std() * 100, 2)}%')

Stop training at epoch: 1500/1500, return best output

MIDA RUN 1/10 - acc: 71.07% +- 2.15% - auc: 77.67% +- 2.71%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 2/10 - acc: 70.73% +- 2.08% - auc: 77.95% +- 2.8%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 3/10 - acc: 71.45% +- 3.39% - auc: 74.96% +- 5.33%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 4/10 - acc: 71.03% +- 3.39% - auc: 76.59% +- 4.29%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 5/10 - acc: 69.26% +- 4.49% - auc: 73.81% +- 3.19%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 6/10 - acc: 70.33% +- 2.58% - auc: 78.12% +- 1.83%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 7/10 - acc: 70.15% +- 4.33% - auc: 77.83% +- 2.89%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 8/10 - acc: 69.82% +- 3.76% - auc: 75.7% +- 2.85%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 9/10 - acc

## Run T-tests

In [12]:
for model, metrics in {
        'MEDIAN': {'ACC': median_accs, 'AUC': median_aucs},
        'MEAN': {'ACC': mean_accs, 'AUC': mean_aucs},
        'KNN': {'ACC': knn_accs, 'AUC': knn_aucs},
        'GAIN': {'ACC': gain_accs, 'AUC': gain_aucs},
        'MICE': {'ACC': mice_accs, 'AUC': mice_aucs},
        'MISS': {'ACC': miss_accs, 'AUC': miss_aucs},
        'SOFT': {'ACC': soft_accs, 'AUC': soft_aucs},
        'SINK': {'ACC': sink_accs, 'AUC': sink_aucs},
        'MIDA': {'ACC': mida_accs, 'AUC': mida_aucs}}.items():
    for metric_name, metric in metrics.items():
        ours_metric = ours_accs if metric_name == 'ACC' else ours_aucs
        t, p = scipy.stats.ttest_ind(np.array(ours_metric), np.array(metric))
        if p <= .05:
            if t > 0:
                ## Our method is better
                print(f'Metric {metric_name} - OUR METHOD is significantly better than {model}')
            else:
                ## Theirs is better
                print(f'Metric {metric_name} - OUR METHOD is significantly worse than {model}')
        else:
            ## Else we are even
            print(f'Metric {metric_name} - OUR METHOD is even with {model}')

Metric ACC - OUR METHOD is significantly better than MEDIAN
Metric AUC - OUR METHOD is significantly better than MEDIAN
Metric ACC - OUR METHOD is significantly worse than MEAN
Metric AUC - OUR METHOD is significantly better than MEAN
Metric ACC - OUR METHOD is significantly better than KNN
Metric AUC - OUR METHOD is significantly better than KNN
Metric ACC - OUR METHOD is significantly better than GAIN
Metric AUC - OUR METHOD is significantly better than GAIN
Metric ACC - OUR METHOD is even with MICE
Metric AUC - OUR METHOD is even with MICE
Metric ACC - OUR METHOD is significantly better than MISS
Metric AUC - OUR METHOD is significantly better than MISS
Metric ACC - OUR METHOD is significantly better than SOFT
Metric AUC - OUR METHOD is significantly better than SOFT
Metric ACC - OUR METHOD is significantly better than SINK
Metric AUC - OUR METHOD is significantly better than SINK
Metric ACC - OUR METHOD is significantly better than MIDA
Metric AUC - OUR METHOD is significantly bett